In [69]:
# import the modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import seaborn as sns


In [70]:
# read the dataset and extract the features and target separately

train = pd.read_csv("train.csv")
train.head()
# train.shape

,Id,Year Type,Year,Organization Group,Department,Union,Job Family,Job,Employee Identifier,Retirement,Health and Dental,Other Benefits,Total Compensation
0,0,Calendar,2018,Community Health,NaN,"Prof & Tech Eng, Local 21",Information Systems,IS Business Analyst-Principal,8594384,$0.0,$13527.66,$0.0,0.00
1,1,Calendar,2014,"Public Works, Transportation & Commerce",MTA Municipal Transprtn Agncy,"SEIU - Miscellaneous, Local 1021",Public Service Aide,Publ Svc Aide-Asst To Prof,22142,$0.0,$3995.56,$875.27,16176.35
2,2,Fiscal,2018,General City Responsibilities,NaN,"SEIU, Local 1021, Misc","Payroll, Billing & Accounting",Senior Account Clerk,8512172,$0.0,$0.0,$0.0,248.30
3,3,Fiscal,2015,Community Health,DPH Public Health,"SEIU - Miscellaneous, Local 1021",Purchasing & Storekeeping,Sr Materials & Supplies Sprv,47593,$11671.59,$10680.55,$4400.24,80020.86
4,4,Fiscal,2014,Community Health,DPH Public Health,"SEIU - Miscellaneous, Local 1021",Hospital Administration,Medical Records Technician,47549,$14617.66,$12464.61,$5562.09,101207.38


In [71]:
train.drop('Id',axis = 1, inplace = True)

In [72]:
train.drop('Employee Identifier', axis = 1,inplace = True)
train.drop('Job', axis = 1, inplace = True)
train.drop('Department', axis = 1, inplace = True)

In [73]:
train.columns

Index(['Year Type', 'Year', 'Organization Group', 'Union', 'Job Family',
       'Retirement', 'Health and Dental', 'Other Benefits',
       'Total Compensation'],
      dtype='object')

In [74]:
train.isnull().sum()

Year Type               0
Year                    0
Organization Group      0
Union                 345
Job Family              0
Retirement              0
Health and Dental       0
Other Benefits          0
Total Compensation      0
dtype: int64

In [75]:

train['Union']=train['Union'].fillna(method ='ffill')

In [76]:
train.isnull().sum()

Year Type             0
Year                  0
Organization Group    0
Union                 0
Job Family            0
Retirement            0
Health and Dental     0
Other Benefits        0
Total Compensation    0
dtype: int64

In [77]:
train.columns=train.columns.str.strip().str.lower().str.replace('$', ' ')

In [78]:
train.columns

Index(['year type', 'year', 'organization group', 'union', 'job family',
       'retirement', 'health and dental', 'other benefits',
       'total compensation'],
      dtype='object')

In [79]:

# Check the percentage of missing value
missing_data = pd.DataFrame({'total_missing': train.isnull().sum(), 'perc_missing': (train.isnull().sum()/train.shape[0])*100})
print(missing_data)

                    total_missing  perc_missing
year type                       0           0.0
year                            0           0.0
organization group              0           0.0
union                           0           0.0
job family                      0           0.0
retirement                      0           0.0
health and dental               0           0.0
other benefits                  0           0.0
total compensation              0           0.0


In [80]:

# mode_1 = train['department'].mode()[0]
# mode_2 = train['job'].mode()[0]
# train['department']=train['department'].replace(np.nan,mode_1)
# train['job']=train['job'].replace(np.nan,mode_2)

In [81]:
train.isnull().sum()

year type             0
year                  0
organization group    0
union                 0
job family            0
retirement            0
health and dental     0
other benefits        0
total compensation    0
dtype: int64

In [82]:
train['other benefits']=train['other benefits'].str.replace('$','')
train['other benefits']

0              0.0
1           875.27
2              0.0
3          4400.24
4          5562.09
            ...   
567098     2888.13
567099     4702.47
567100         0.0
567101         0.0
567102    10965.64
Name: other benefits, Length: 567103, dtype: object

In [83]:

# Removing `$` from the column retirement
train['retirement']=train['retirement'].str.replace('$','')

# Removing `$` from the column health and dental
train['health and dental']=train['health and dental'].str.replace('$','')

train[['retirement','health and dental','other benefits']]


,retirement,health and dental,other benefits
0,0.0,13527.66,0.0
1,0.0,3995.56,875.27
2,0.0,0.0,0.0
3,11671.59,10680.55,4400.24
4,14617.66,12464.61,5562.09
...,...,...,...
567098,9483.37,8487.84,2888.13
567099,12963.3,7774.8,4702.47
567100,0.0,4617.42,0.0
567101,0.0,13037.64,0.0


In [84]:
train['retirement'] = train['retirement'].astype(float)
train['retirement'] = train['retirement'].astype(int)

In [85]:
train[['retirement','health and dental','other benefits']]

,retirement,health and dental,other benefits
0,0,13527.66,0.0
1,0,3995.56,875.27
2,0,0.0,0.0
3,11671,10680.55,4400.24
4,14617,12464.61,5562.09
...,...,...,...
567098,9483,8487.84,2888.13
567099,12963,7774.8,4702.47
567100,0,4617.42,0.0
567101,0,13037.64,0.0


In [86]:
train['health and dental'] = train['health and dental'].astype(float)
train['health and dental'] = train['health and dental'].astype(int)

In [87]:
train['other benefits'] = train['other benefits'].astype(float)
train['other benefits'] = train['other benefits'].astype(int)

In [88]:
train[['retirement','health and dental','other benefits']]


,retirement,health and dental,other benefits
0,0,13527,0
1,0,3995,875
2,0,0,0
3,11671,10680,4400
4,14617,12464,5562
...,...,...,...
567098,9483,8487,2888
567099,12963,7774,4702
567100,0,4617,0
567101,0,13037,0


In [89]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_cat =train[train.select_dtypes('object').columns]
df_cat

for col in df_cat:
    df_cat[col]=le.fit_transform(df_cat[col])

df_cat.head()

C:\Users\sarika\python\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,year type,organization group,union,job family
0,0,0,78,23
1,0,6,90,46
2,1,3,93,35
3,1,0,90,47
4,1,0,90,20


In [90]:
train['retirement']=le.fit_transform(train['retirement'])

In [91]:
train['health and dental']=le.fit_transform(train['health and dental'])

In [92]:
train['other benefits']=le.fit_transform(train['other benefits'])

In [93]:
train[['retirement','health and dental','other benefits']]


,retirement,health and dental,other benefits
0,147,13514,152
1,147,4020,1027
2,147,33,152
3,11806,10667,4552
4,14752,12451,5714
...,...,...,...
567098,9618,8500,3040
567099,13098,7788,4854
567100,147,4641,152
567101,147,13024,152


In [94]:
df_cat

,year type,organization group,union,job family
0,0,0,78,23
1,0,6,90,46
2,1,3,93,35
3,1,0,90,47
4,1,0,90,20
...,...,...,...,...
567098,1,5,67,37
567099,1,2,82,7
567100,0,5,35,18
567101,0,4,94,33


In [95]:
df_num = train.select_dtypes(include=['int64', 'float64'])
df_num

,year,retirement,health and dental,other benefits,total compensation
0,2018,147,13514,152,0.00
1,2014,147,4020,1027,16176.35
2,2018,147,33,152,248.30
3,2015,11806,10667,4552,80020.86
4,2014,14752,12451,5714,101207.38
...,...,...,...,...,...
567098,2018,9618,8500,3040,74977.10
567099,2014,13098,7788,4854,86244.18
567100,2018,147,4641,152,0.00
567101,2017,147,13024,152,0.00


In [96]:
new_dataframe= pd.concat([df_cat,df_num],axis=1)


new_dataframe.head()

# new_dataframe.shape


,year type,organization group,union,job family,year,retirement,health and dental,other benefits,total compensation
0,0,0,78,23,2018,147,13514,152,0.00
1,0,6,90,46,2014,147,4020,1027,16176.35
2,1,3,93,35,2018,147,33,152,248.30
3,1,0,90,47,2015,11806,10667,4552,80020.86
4,1,0,90,20,2014,14752,12451,5714,101207.38


In [97]:
new_dataframe.isnull().sum()

year type             0
organization group    0
union                 0
job family            0
year                  0
retirement            0
health and dental     0
other benefits        0
total compensation    0
dtype: int64

In [98]:
new_dataframe.columns

Index(['year type', 'organization group', 'union', 'job family', 'year',
       'retirement', 'health and dental', 'other benefits',
       'total compensation'],
      dtype='object')

In [99]:
# Code Starts here
X =  new_dataframe.drop('total compensation', axis = 1)

y = new_dataframe['total compensation']
# Separate into train and test data
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3)

# print(X_train.head(2))
# print(y_train.head(2))

# Instantiate linear regression
model = LinearRegression()

# fit the model on training data
model.fit(X_train,y_train)


# make prediction
y_pred = model.predict(X_test)

# calculate the mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print(mae)

# calculate the r2 score
r2 = r2_score(y_test, y_pred)
print(r2) 


# Code ends here

11234.43282349406
0.9375634438815398


### Prediction on the test data and creating the sample submission file.

- Load the test data and store the `Id` column in a separate variable.
- Perform the same operations on the test data that you have performed on the train data.
- Create the submission file as a `csv` file consisting of the `Id` column from the test data and your prediction as the second column.

In [100]:
test=pd.read_csv("test.csv")

In [101]:
test.head()

,Id,Year Type,Year,Organization Group,Department,Union,Job Family,Job,Employee Identifier,Retirement,Health and Dental,Other Benefits
0,0,Calendar,2017,"Public Works, Transportation & Commerce",NaN,"Electrical Workers, Local 6",Journeyman Trade,Electrical Line Worker,8549078,$0.0,$13051.16,$0.0
1,1,Calendar,2019,Public Protection,NaN,"SEIU, Local 1021, Misc",Correction & Detention,"Counselor, Juvenile Hall PERS",8546443,$21487.97,$28361.72,$753.13
2,2,Fiscal,2018,Community Health,NaN,"SEIU, Local 1021, Misc",Dietary & Food,Food Service Worker,8512207,$12468.31,$13863.98,$4464.84
3,3,Fiscal,2018,Public Protection,NaN,"SEIU, Local 1021, Misc",Correction & Detention,"Counselor, Juvenile Hall SFERS",8542154,$0.0,$0.0,$1787.63
4,4,Fiscal,2017,Public Protection,POL Police,"SEIU - Miscellaneous, Local 1021",Protection & Apprehension,Police Cadet,48828,$4489.2,$8003.53,$2015.35


In [102]:
# test.drop('Id',axis = 1, inplace = True)
test.drop('Employee Identifier', axis = 1,inplace = True)
test.drop('Job', axis = 1, inplace = True)
test.drop('Department', axis = 1, inplace = True)

In [103]:
test.isnull().sum()

Id                     0
Year Type              0
Year                   0
Organization Group     0
Union                 95
Job Family             0
Retirement             0
Health and Dental      0
Other Benefits         0
dtype: int64

In [104]:
# mode_1 = test['Department'].mode()[0]
# mode_2 = test['Job'].mode()[0]
# test['Department']=test['Department'].replace(np.nan,mode_1)
# test['Job']=test['Job'].replace(np.nan,mode_2)
test['Union']=test['Union'].fillna(method ='ffill')

In [105]:
test.isnull().sum()

Id                    0
Year Type             0
Year                  0
Organization Group    0
Union                 0
Job Family            0
Retirement            0
Health and Dental     0
Other Benefits        0
dtype: int64

In [106]:
le=LabelEncoder()

test['Retirement']=test['Retirement'].str.replace('$','')
test['Retirement'] = test['Retirement'].astype(float)
test['Retirement'] = test['Retirement'].astype(int)

test['Retirement']=le.fit_transform(test['Retirement'])


In [107]:
# Removing `$` from the column health and dental
test['Health and Dental']=test['Health and Dental'].str.replace('$','')
test['Health and Dental'] = test['Health and Dental'].astype(float)
test['Health and Dental'] = test['Health and Dental'].astype(int)

# Removing `$` from the column other benefits
test['Other Benefits']=test['Other Benefits'].str.replace('$','')

test['Other Benefits'] = test['Other Benefits'].astype(float)
test['Other Benefits'] = test['Other Benefits'].astype(int)



test['Other Benefits']=le.fit_transform(test['Other Benefits'])
test['Health and Dental']=le.fit_transform(test['Health and Dental'])


In [108]:

test[['Retirement','Health and Dental','Other Benefits']]

,Retirement,Health and Dental,Other Benefits
0,47,11942,63
1,20609,16254,816
2,11823,12728,4527
3,47,12,1850
4,4128,7295,2078
...,...,...,...
177215,14342,7492,6330
177216,5467,4180,2279
177217,5776,6151,2238
177218,11419,11394,3963


In [109]:
df_num = test.select_dtypes(include=['int64', 'float64'])
df_num

,Id,Year,Retirement,Health and Dental,Other Benefits
0,0,2017,47,11942,63
1,1,2019,20609,16254,816
2,2,2018,11823,12728,4527
3,3,2018,47,12,1850
4,4,2017,4128,7295,2078
...,...,...,...,...,...
177215,177215,2015,14342,7492,6330
177216,177216,2014,5467,4180,2279
177217,177217,2020,5776,6151,2238
177218,177218,2015,11419,11394,3963


In [110]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_cat =test[test.select_dtypes('object').columns]
df_cat

for col in df_cat:
    df_cat[col]=le.fit_transform(df_cat[col])

df_cat.head()

C:\Users\sarika\python\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Year Type,Organization Group,Union,Job Family
0,0,6,31,24
1,0,5,93,13
2,1,0,93,14
3,1,5,93,13
4,1,5,90,42


In [111]:
new_dataframe1= pd.concat([df_cat,df_num],axis=1)


new_dataframe1.columns

Index(['Year Type', 'Organization Group', 'Union', 'Job Family', 'Id', 'Year',
       'Retirement', 'Health and Dental', 'Other Benefits'],
      dtype='object')

In [113]:
new_dataframe1.columns

Index(['Year Type', 'Organization Group', 'Union', 'Job Family', 'Id', 'Year',
       'Retirement', 'Health and Dental', 'Other Benefits'],
      dtype='object')

In [114]:
# Prediction on test data

# Read the test data
test = pd.read_csv('C:/Users/sarika/python practice/EDA prework/test.csv')

# Storing the id from the test file 
id_ = new_dataframe1['Id']

# # Applying same transformation on test
# new_dataframe1.drop('Id',axis=1,inplace=True)

new_dataframe1 =  new_dataframe1[['Year Type', 'Organization Group', 'Union', 'Job Family', 'Year',
       'Retirement', 'Health and Dental', 'Other Benefits']]

# # make predictions 
y_pred_test = model.predict(new_dataframe1)

# # Create a sample submission file
sample_submission = pd.DataFrame({'Id':id_,'Total Compensation':y_pred_test})

# # Convert the sample submission file into a csv file
sample_submission.to_csv('sample_submission4.csv',index=False)